# 03 Dictionaries and Sets
Some notes, observations and questions along chapter 03.

[Internals of sets and dicts](https://www.fluentpython.com/extra/internals-of-sets-and-dicts/) explains how dicts and sets are build on top of hash tables and what that means.

### Modern dict Syntax

In [2]:
# dict unpacking with **
def dump(**kwargs):
    return kwargs

dump(**{'x': 1}, y=2, **{'z': 3}) # it's unpacking and packing in the same step

{'x': 1, 'y': 2, 'z': 3}

In [3]:
# Merging Mappings with | and |= (the pipe is an union operator)
d1 = {'a': 1, 'b': 3}
d2 = {'a': 2, 'b': 4, 'c': 6}
d1 | d2 # note when same key, the later values replace the earlier ones

{'a': 2, 'b': 4, 'c': 6}

### Pattern Matching with Mappings

-especially useful for communicating with databases and JSON files (which are similar to dicts)

### Standard API of Mapping Types

- from Python glossary about what is hasable:

"An object is hashable if it has a hash code which never changes during its lifetime (it needs a __hash__() method), and can be compared to other objects (it needs an __eq__() method). Hashable objects which compare equal must have the same hash code."

- Numeric types and flat immutable types str and bytes are all hashable. 
- Container types are hashable if they are immutable and all contained objects are also hashable.

### Automatic Handling of Missing Keys

defaultdict: Another Take on Missing Keys

- creates items with a default value on demand whenever a missing key is searched using d[k] syntax

In [35]:
from collections import defaultdict

d = defaultdict(list) # list is the default value
d

defaultdict(list, {})

In [8]:
d['a'] # creating a key 'a' that has the default value

[]

In [10]:
d # key and default value are permanently part of the object

defaultdict(list, {'a': []})

"when instantiating a defaultdict, you provide a callable to produce a default value whenever __getitem__ is passed a nonexistent key argument."

- list is a callable, because then list() will be called if a key is not found (question: is any class instantiation at the same time an act of calling a callable?)
- this callable is held in an instance attribute called "default_factory":

In [13]:
d.default_factory

list

The `default_factory` attribute and the use of a `__missing__` method are the only differences between a regular dict and defaultdict. We can also only extend on a dict. When we implement a `__missing__` method on a class subclassing dict, then the `instance.__getitem__` will automatically call the `__missing__` method instead or raising a KeyError.

In [32]:
class StrKeyDict0(dict):

    def __missing__(self, key): # called from within __getitem__ if key is missing
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def get(self, key, default=None): 
        # The get method delegates to __getitem__ by using the self[key] notation; 
        # that gives the opportunity for our __missing__ to act.
        try:
            return self[key]
        except KeyError:
            return default

In [33]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])

# these examples use __getitem__ right away and not get():
d['2'] # returns 'two'
d[4] # returns 'four'
d["three"] # KeyError

KeyError: 'three'

But with get() we can define a default value when the key is not found:

In [34]:
d.get('2')  # Returns 'two'
d.get(4)    # Returns 'four' (converts 4 to '4' and finds it)
d.get(1, 'N/A')  # Returns 'N/A' (since '1' is not in the dictionary); default would be None

'N/A'

### Variations of dict
collections.OrderedDict
- only used to keep backwards compatibility with older Python versions (since dict in an ordered type since Python 3.6)

collections.ChainMap
- used for lookup a value under nested conditions
- "A ChainMap instance holds a list of mappings that can be searched as one. It does not copy the input mappings, but holds references to them."

In [5]:
d1 = dict(a=1, b=3)
d2 = dict(a=2, b=4, c=6)

from collections import ChainMap
chain = ChainMap(d1, d2)
chain['a'] # note that the instance from the first mapping is returned 
# and the second one is ignored (what would this be useful for?)

1

In [2]:
chain['c']

6

Updates only affect the first chained dict instance:

In [3]:
chain['c'] = -1
d1

{'a': 1, 'b': 3, 'c': -1}

In [4]:
d2

{'a': 2, 'b': 4, 'c': 6}

collections.Counter
- "A mapping that holds an integer count for each key."
- "Updating an existing key adds to its count."
- has some special methods implemended like + and - and `most_common([n])`

In [13]:
# Counter to count letters
from collections import Counter

ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [14]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'z': 3, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [16]:
ct.most_common(3) # "b" and "r" are both in third place, but most_common only shows the three first positions

[('a', 10), ('z', 3), ('b', 2)]

The book also provides another use of Counter: "To use collections.Counter as a multiset, pretend each key is an element in the set, and the count is the number of occurrences of that element in the set."
- to me though this seems the be same as the common use case
- what is the difference? 

shelve.Shelf
- "The shelve module in the standard library provides persistent storage for a mapping of string keys to Python objects serialized in the pickle binary format."

Subclassing UserDict Instead of dict
- `UserDict` is designed to be subclassed, not the be uses as is.
- this is almost always preferred in contrast to subclassing `dict` when creating a new mapping class
- "The main reason why it’s better to subclass UserDict rather than dict is that the built-in has some implementation shortcuts that end up forcing us to override methods that we can just inherit from UserDict with no problems."

### Immutable Mappings
- if we want to prevent a mapping to be changed, we can use `MappingProxyType` from the `type` module: wrapping it around a mapping returns a `mappingproxy` instance that is a read-only version of the mapping; updates in the original mapping can be seen in the `mappingproxi` instance

### Dictionary Views
- views allow high-performance operations on a dict, without unnecessary copying of data.
- "The dict instance methods .keys(), .values(), and .items() return instances of classes called dict_keys, dict_values, and dict_items, respectively. These dictionary views are read-only projections of the internal data structures used in the dict implementation."

In [17]:
d = dict(a=10, b=20, c=30)
values = d.values()
values

dict_values([10, 20, 30])

In [19]:
# a view object is a dynamic proxy:
d['z'] = 99
values

dict_values([10, 20, 30, 99])

The dict_values class is the simplest dictionary view—it implements only the __len__, __iter__, and __reversed__ special methods.

### Practical Consequences of How dict Works
- this section goes into hash tables, but without explaining what it is, and I would like to find some material 
that introduces hash tables and their use cases

### Set Theory
- good performance and easy to read thanks to infix operators (`|`, `&`, `^`, `\`)
- for instance `found = len(needles & haystack)`
- we can also do set comprehensions:

In [25]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

The chr() method converts an integer to its unicode character and returns it.

### Practical Consequences of How Sets Work

- since a set (like a dict) is build on top of a hash table, a set may have millions of elements, but an element can be located directly by computing its hash code and deriving an index offset.
- sets have a significant memory overhead, but are fast in execution (time complexity)

### Set operations on dictionary views
- dict vies like d.keys() and d.values() have the same operations implemented like frozenset, especially & (intersection), | (union), - (difference), and ^ (symmetric difference).

In [36]:
d1 = dict(a=1, b=2, c=3, d=4)
d2 = dict(b=20, d=40, e=50)
d1.keys() & d2.keys()

{'b', 'd'}

Note that the return value is a set.

The set operators in dictionary views are compatible with set instances:

In [37]:
s = {'a', 'e', 'i'}
d1.keys() | s

{'a', 'b', 'c', 'd', 'e', 'i'}